In [ ]:
from model import MAE
import torch
from utils import train, test, tdlr, vdlr, get_cosine_schedule_with_warmup

model = MAE()
model.to('cuda')
torch.compile(model)

In [ ]:
max_lr = 1e-4
min_lr = 0.1 * max_lr
epochs = 100
epoch_steps = len(tdlr)
gradient_step = 4
max_steps = (epoch_steps * epochs)  / gradient_step
warmup_steps = (max_steps * 10) // 100
optimizer = torch.optim.AdamW(model.parameters(), lr = min_lr, eps=1e-8)
sch = get_cosine_schedule_with_warmup(optimizer, warmup_steps, max_steps)
loss_fn = torch.nn.MSELoss(reduction='sum')

In [ ]:
for epoch in range(epochs):
    
    train(model, optimizer, sch, loss_fn, epoch, tdlr)
    test(model, loss_fn, epoch, vdlr)

torch.save(model.state_dict(), 'output.lol')